In [8]:
import pickle
from sklearn_crfsuite import metrics
from itertools import chain

with open('input_kb/labels_enesub.txt') as f:
    LABELS = f.read().split('\n')

class Sentence:

    def __init__(self, sentence_str: str):
        self.entries = []
        self.labels = []
        for line in sentence_str.split('\n'):
            entry = line.split(' ')[:-1]
            label = line.split(' ')[-1]
            if not entry:
                continue
            self.entries.append(entry)
            self.labels.append(label)

        self.n = len(self.labels)

def read_data(filepath):
    with open(filepath) as f:
        return [Sentence(sentence_str) for sentence_str in f.read().split('\n\n')]


In [9]:
# CRFベースライン (学習10分)

with open('crfresult_test_baseline.pkl', 'rb') as f:
    result = pickle.load(f)
len(result)
sentences_test = read_data('input_kb/test.txt')  #'NERdata_ene_single_cv_5/NERdata_0/test.txt')
y_pred = result
y_test = [s.labels for s in sentences_test]


print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted(LABELS), digits=3
))

ValueError: Found input variables with inconsistent numbers of samples: [114358, 118705]

In [39]:
# CRF+Gazetteer素性(multi) (学習8分)

from crf_entity_extractor import EntityExtractor



with open('crfresult_test_gazetter_multi.pkl', 'rb') as f:
    result = pickle.load(f)
len(result)
sentences_test = read_data('input_kb/test.txt')  #'NERdata_ene_single_cv_5/NERdata_0/test.txt')
y_pred = result
y_test = [s.labels for s in sentences_test]

ee = EntityExtractor()
ee.load_model('crf_gazetter_multi.pkl')
score_result = ee.evaluate(sentences_test)

# print(metrics.flat_classification_report(
#     y_test, y_pred, labels=sorted(LABELS), digits=3
# ))

/Users/kzinmr/.local/share/virtualenvs/bert-ner-ja-BS2OIZ5b/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kzinmr/.local/share/virtualenvs/bert-ner-ja-BS2OIZ5b/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


                                precision    recall  f1-score   support

                     B-Airport      1.000     0.600     0.750         5
              B-Amusement_Park      1.000     0.545     0.706        11
                     B-Cabinet      1.000     0.500     0.667         4
                    B-Car_Stop      0.000     0.000     0.000         0
                        B-City      0.856     0.646     0.736       506
                     B-Company      0.804     0.478     0.600       318
               B-Company_Group      0.667     0.483     0.560        29
           B-Corporation_Other      0.708     0.508     0.592       124
                     B-Country      0.930     0.892     0.911       825
                      B-County      0.600     0.600     0.600         5
          B-Ethnic_Group_Other      0.806     0.625     0.704        40
                      B-Family      1.000     0.353     0.522        17
                   B-GOE_Other      0.673     0.285     0.400  

/Users/kzinmr/.local/share/virtualenvs/bert-ner-ja-BS2OIZ5b/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/kzinmr/.local/share/virtualenvs/bert-ner-ja-BS2OIZ5b/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
Original:
                     micro avg      0.843     0.669     0.746     12852
                     macro avg      0.775     0.473     0.566     12852
                  weighted avg      0.840     0.669     0.736     12852
After fixing invalid tags:
                     micro avg      0.850     0.668     0.748     12852
                     macro avg      0.778     0.472     0.566     12852
                  weighted avg      0.843     0.668     0.737     12852

In [5]:
# CRF+Gazetteer素性(multi-labeled) (学習8分)

with open('crfresult_test_gazetter_multi.pkl', 'rb') as f:
    result = pickle.load(f)
len(result)
sentences_test = read_data('NERdata_ene_single_cv_5/NERdata_0/test.txt')
y_pred = result
y_test = [s.labels for s in sentences_test]


print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted(LABELS), digits=3
))

                                precision    recall  f1-score   support

                     B-Airport      1.000     0.714     0.833         7
              B-Amusement_Park      0.875     0.467     0.609        15
                     B-Cabinet      0.833     0.833     0.833         6
                    B-Car_Stop      1.000     0.200     0.333         5
                        B-City      0.852     0.742     0.794       466
                     B-Company      0.806     0.513     0.627       388
               B-Company_Group      0.526     0.435     0.476        23
           B-Corporation_Other      0.744     0.616     0.674       151
                     B-Country      0.904     0.916     0.910       774
                      B-County      0.000     0.000     0.000         4
          B-Ethnic_Group_Other      0.879     0.527     0.659        55
                      B-Family      1.000     0.435     0.606        23
                   B-GOE_Other      0.712     0.280     0.402  